# <center> Проект-2: Анализ вакансий из HeadHunter

In [672]:
import psycopg2
import pandas as pd

import warnings
warnings.filterwarnings('ignore') # скрываем предупреждения

In [674]:
# Создаём соединение с заданными параметрами:
connection = psycopg2.connect(
   dbname=DBNAME,
   user=USER,
   host=HOST,
   password=PASSWORD,
   port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишем запрос, который посчитает количество вакансий в базе.

In [675]:
query = f'''select *
            from public.VACANCIES           
        '''
df = pd.read_sql_query(query, connection)
print('Количество вакансий в базе VACANCIES равно: ', df.shape[0])

Количество вакансий в базе VACANCIES равно:  49197


2. Напишем запрос, который посчитает количество работодателей в базе.

In [676]:
query = f'''select *
            from public.EMPLOYERS           
        '''
df = pd.read_sql_query(query, connection)
print('Количество работодателей равно: ', df.shape[0])

Количество работодателей равно:  23501


3. Посчитаем с помощью запроса количество регионов.

In [677]:
query = f'''select *
            from public.AREAS           
        '''
df = pd.read_sql_query(query, connection)
print('Количество регионов: ', df.shape[0])

Количество регионов:  1362


4. Посчитаем с помощью запроса количество сфер деятельности в базе.

In [678]:
query = f'''select *
            from public.INDUSTRIES           
        '''
df = pd.read_sql_query(query, connection)
print('Количество сфер деятельности: ', df.shape[0])

Количество сфер деятельности:  294


***
Выводы по Юниту 3.  

Согласно полученным данным, в 1362 регионах зарегистрировано порядка 23,5 тыс. работодателей (в различных сферах деятельности) и открыто более 49 тыс. вакансий. 
Это значит, что  в среднем, работодатель открывает более одной вакансии. 
Можно предположить, что распределение работодателей и вакансий будет неравномерным по регионам и будет смещено к большим городам/регионам (Моска, Санкт-Петербург и т.д.)

# Юнит 4. Детальный анализ вакансий

1. Напишем запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area). Отсортируем по количеству вакансий в порядке убывания.

In [679]:
query = f'''select 
                a.name area,
                count(v.id) cnt
            from public.AREAS as a
                left join public.VACANCIES as v on a.id = v.area_id
            group by a.name
            order by 2 desc
            --limit 5           
        '''
df = pd.read_sql_query(query, connection)
print('Количество вакансий по регионам:')
display(df)

Количество вакансий по регионам:


,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
1357,Багаевская,0
1358,Пролетарск,0
1359,Седельниково,0
1360,Инской,0


2. Напишем запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [680]:
query = f'''select 
                count(v.id)
            from public.VACANCIES as v
            where v.salary_from is not NULL or v.salary_to is not NULL          
        '''
df = pd.read_sql_query(query, connection)
print("Количество вакансий, у которых заполнено хотя бы одно из двух полей с зарплатой: ", df.iloc[0,0])

Количество вакансий, у которых заполнено хотя бы одно из двух полей с зарплатой:  24073


3. Найдем средние значения для нижней и верхней границы зарплатной вилки. Округлим значения до целого числа.

In [681]:
query = f'''select 
                round(avg(v.salary_from)),
                round(avg(v.salary_to))
            from public.VACANCIES as v 
        '''
df = pd.read_sql_query(query, connection)
print("Среднее значение нижней границы зарплатной вилки: ", df.iloc[0,0])
print("Среднее значение верхней границы зарплатной вилки: ", df.iloc[0,1])

Среднее значение нижней границы зарплатной вилки:  71065.0
Среднее значение верхней границы зарплатной вилки:  110537.0


4. Напишем запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. 

In [682]:
query = f'''select                                            
                v.schedule,
                v.employment,
                count(v.id)
            from public.VACANCIES as v            
            group by v.schedule, v.employment   
            order by 3 desc                          
        '''
df = pd.read_sql_query(query, connection)
display(df)
print('Всего вакансий: ', sum(df['count']))

,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


Всего вакансий:  49197


5. Напишем запрос, выводящий значения поля *Требуемый опыт работы* (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта.

In [683]:
query = f'''select                      
                v.experience,
                count(v.id)
            from public.VACANCIES as v            
            group by v.experience           
            order by 2
        '''
df = pd.read_sql_query(query, connection)
display(df)

,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


*** 
Выводы по Юниту 4.

По количеству открытых вакансий лидируют крупные города-миллионники. Около половины работодателей указывают зарплатную вилку, или обозначают границы возможного заработка: "от"/"до". 
Очевидно, что в формирование таких показателей как среднее значение нижней и верхней границы зарплатной вилки (71 тыс. и 110 тыс. соответственно) свой вес внесли регионы Москва и Санкт-Петербург. Для сравнения, средняя з/п по России в 2022 году составила 53 тыс. рублей.
В основном, работодатели ищут сотрудников на полный день и полную занятость ("работа в офисе") - около 72% вакансий ($\frac{35367*100\%}{49197}$). Но так же немало вакансий предполагают удаленный формат работы - около 16% ($\frac{7802*100\%}{49197}$). Абсолютное большинство вакансий рассчитано на соискателей с уже имеющимся опытом работы 1-3 года. Можно предположить, что это связано, с появлением достаточных знаний и компетенций для начала работы в компании, при этом такие сотрудники как правило претендуют на более низкую зарплату, по сравнению с более опытными коллегами. 

# Юнит 5. Анализ работодателей

1. Напишем запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [684]:
query = f'''select      
                e.name,
                count(v.id)
            from public.EMPLOYERS as e            
                join public.VACANCIES as v on e.id = v.employer_id
            group by e.name
            order by 2 desc
            limit 5
        '''
df = pd.read_sql_query(query, connection)
display(df)

,name,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишем запрос, который для каждого региона выведет количество работодателей и вакансий в нём. Среди регионов, в которых нет вакансий, найдем тот, в котором наибольшее количество работодателей.


In [685]:
query = f'''select                            
                a.name,
                count(e.id) "Кол-во работодателей",
                count(v.id) "Кол-во вакансий"
            from public.AREAS as a           
                    left join public.EMPLOYERS as e on a.id=e.area 
                    left join public.VACANCIES as v on a.id = v.area_id            
            where v.id is null
            group by a.name              
            order by 3 NULLS FIRST, 2 desc            
        '''
df = pd.read_sql_query(query, connection)
display(df)

,name,Кол-во работодателей,Кол-во вакансий
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Беларусь,18,0
...,...,...,...
588,Кувандык,0,0
589,Куйбышев,0,0
590,Куйбышево (Ростовская область),0,0
591,Култаево,0,0


3. Для каждого работодателя посчитаем количество регионов, в которых он публикует свои вакансии. Отсортируем результат по убыванию количества.

In [686]:
query = f'''select                
                e.name,
                count(distinct v.area_id) "Количество регионов"                
            from public.EMPLOYERS as e           
                left join public.VACANCIES as v on e.id = v.employer_id
            group by e.name
            order by 2 desc            
        '''
df = pd.read_sql_query(query, connection)
display(df)

,name,Количество регионов
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
23170,СДЕЛКА,0
23171,Alandr Group,0
23172,СДК,0
23173,Lemon Land Lombard,0


4. Напишем запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [687]:
query = f'''select               
                count(distinct e.id)
            from public.EMPLOYERS as e           
                left join public.EMPLOYERS_INDUSTRIES as emp on e.id = emp.employer_id
                left join public.INDUSTRIES as i on emp.industry_id=i.id            
            where i.name is NULL            
        '''
df = pd.read_sql_query(query, connection)
print("Количество работодателей, у которых не указана сфера деятельности: ", df.iloc[0,0])

Количество работодателей, у которых не указана сфера деятельности:  8419


5. Напишем запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [688]:
query = f'''select                
                e.name                
            from public.EMPLOYERS as e           
                join public.EMPLOYERS_INDUSTRIES as emp on e.id = emp.employer_id
            group by e.name
            having  count(emp.industry_id)=4 
            order by 1            
        '''
df = pd.read_sql_query(query, connection)
display(df)

,name
0,101 Интернет
1,21vek.by
2,2ГИС
3,2К
4,4 пикселя +
...,...
1131,ЮРИОН
1132,ЮТИП Технологии
1133,ЯКласс
1134,ЯрНео


6. С помощью запроса выясним, у какого количества работодателей в качестве сферы деятельности указана «Разработка программного обеспечения».

In [689]:
query = f'''select               
                count(i.id)                
            from public.INDUSTRIES as i  
                join public.EMPLOYERS_INDUSTRIES as emp on i.id = emp.industry_id
                join public.EMPLOYERS as e on emp.employer_id = e.id           
            where i.name='Разработка программного обеспечения'            
        '''
df = pd.read_sql_query(query, connection)
print("Кол-во кработодателей, у которых в качестве сферы деятельности указана Разработка ПО: ", df.iloc[0,0])

Кол-во кработодателей, у которых в качестве сферы деятельности указана Разработка ПО:  3553


7. Для компании «Яндекс» выведем список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах.

In [690]:
import urllib.parse as req
path_common = req.urlparse('https://ru.wikipedia.org/wiki/Города-миллионеры_России#Список_городов-миллионеров') 
url = path_common.scheme + '://' + req.quote(path_common.netloc) + req.quote(path_common.path) + '#'+ req.quote(path_common.fragment) 
df1 = pd.read_html(url)[1]
df_sity=tuple(df1["Город"])
#display(df_sity)

query = f'''select               
                a.name,
                count(v.id) cnt
            from public.EMPLOYERS as e  
                left join public.VACANCIES as v on e.id = v.employer_id
                left join public.areas as a on v.area_id = a.id                         
            where e.name='Яндекс' and (a.name in {df_sity} or a.name = 'Воронеж')        
            group by a.name                
            union
            select
                'Total', sum(v.id/v.id)  
            from public.EMPLOYERS as e  
                left join public.VACANCIES as v on e.id = v.employer_id 
                left join public.areas as a on v.area_id = a.id                         
            where e.name='Яндекс' and (a.name in {df_sity} or a.name = 'Воронеж')
               
            order by 2
        '''
df = pd.read_sql_query(query, connection)
display(df)


,name,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***
Выводы по Юниту 5.

Лидирующие позиции по количеству вакансий занимают такие крупные компании как Яндекс, Ростелеком, Тинькофф и проч., которые имеют филиалы по всей России и многим странам ближнего зарубежья, эти компании так же могут позволить себе нанимать сотрудников на "удаленке", что так же способствует увеличению кол-ва открытых вакансий.
Чем крупнее регион, тем больше работодателей зарегистрировано в нем и тем большее количество вакансий в нем открыто.
Большинство работодателей указывает свою сферу деятельности (одну и более), а не указана она только у 35% работодателей ($\frac{8419*100\%}{23501}$). Около 15% работодателей имеют отношение к разработке ПО.

# Юнит 6. Предметный анализ

1. Подсчитаем, сколько вакансий имеет отношение к данным (в её названии содержатся слова 'data' или 'данн').

In [691]:
query = f'''select              
                count(v.id)                
            from public.VACANCIES as v           
            where v.name ilike '%data%' or v.name ilike '%данн%'            
        '''
df = pd.read_sql_query(query, connection)
print("Количество вакансий, имеющих отношение к данным: ", df.iloc[0,0])

Количество вакансий, имеющих отношение к данным:  1771


2. Посчитаем, сколько есть подходящих вакансий для начинающего дата-сайентиста.

In [692]:
query = f'''select               
                count(v.name)                
            from public.VACANCIES as v                                 
            where (v.name ilike '%data scientist%' or v.name ilike '%data science%' or v.name ilike '%исследователь данных%' 
                or (v.name like '%ML%' and v.name not ilike '%HTML%') or v.name ilike '%machine learning%' 
                or v.name ilike '%машинн%обучен%') 
                and (v.name ilike '%junior%' or v.experience ilike '%нет опыта%' or v.employment ilike '%стажировка%')            
        '''
df = pd.read_sql_query(query, connection)
print("Количество подходящих вакансий для начинающего дата-сайентиста: ", df.iloc[0,0])

Количество подходящих вакансий для начинающего дата-сайентиста:  51


3. Посчитаем, сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres.

In [693]:
query = f'''select               
                count(*)  

            from public.VACANCIES as v                                 
            
            where (v.name ilike '%data scientist%' or v.name ilike '%data science%' or v.name ilike '%исследователь данных%' 
                or (v.name like '%ML%' and v.name not ilike '%HTML%') or v.name ilike '%machine learning%' or v.name ilike '%машинн%обучен%') 
                and (v.key_skills ilike '%SQL%' or v.key_skills ilike '%postgres%')           
        '''
df = pd.read_sql_query(query, connection)
print("Количество вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres: ", df.iloc[0,0])

Количество вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres:  201


4. Проверим, насколько популярен Python в требованиях работодателей к DS. Для этого вычислим количество вакансий, в которых в качестве ключевого навыка указан Python.

In [694]:
query = f'''select               
                count(*)                
            from public.VACANCIES as v                                 
            
            where ((v.name ilike '%data scientist%') or (v.name ilike '%data science%') or (v.name ilike '%исследователь данных%') 
                or ((v.name like '%ML%') and (v.name not ilike '%HTML%')) or (v.name ilike '%machine learning%') or (v.name ilike '%машинн%обучен%')  
                and (v.key_skills is not null)) and v.key_skills ilike '%python%'               
        '''
df = pd.read_sql_query(query, connection)
print("Количество вакансий, в которых в качестве ключевого навыка указан Python: ", df.iloc[0,0])

Количество вакансий, в которых в качестве ключевого навыка указан Python:  351


5. Посчитаем, сколько ключевых навыков в среднем указывают в вакансиях для DS? Ответ округлим до двух знаков после точки-разделителя.

In [695]:
query = f'''select               
                round(avg(length(v.key_skills) - length( replace(v.key_skills, CHR(9), '')) + 1),2) as count_skills
                
            from public.VACANCIES as v                                 
            
            where (v.name ilike '%data scientist%') or (v.name ilike '%data science%') or (v.name ilike '%исследователь данных%') 
                or ((v.name like '%ML%') and (v.name not ilike '%HTML%')) or (v.name ilike '%machine learning%') or (v.name ilike '%машинн%обучен%')
                and (v.key_skills is not NULL)            
        '''
df = pd.read_sql_query(query, connection)
print("Среднее значение количества ключевых навыков в вакансиях для DS: ", df.iloc[0,0])

Среднее значение количества ключевых навыков в вакансиях для DS:  6.41


6. Напишем запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*).

In [696]:
query = f'''select               
                v.experience,
                round(avg((coalesce(v.salary_from, v.salary_to)+coalesce(v.salary_to, v.salary_from))/2)) salary

            from public.VACANCIES as v                                 
            
            where ((v.name ilike '%data scientist%') or (v.name ilike '%data science%') or (v.name ilike '%исследователь данных%') 
                or ((v.name like '%ML%') and (v.name not ilike '%HTML%')) or (v.name ilike '%machine learning%') or (v.name ilike '%машинн%обучен%'))
                and (v.salary_from is not NULL or v.salary_to is not NULL)
                
            group by v.experience           
        '''
df = pd.read_sql_query(query, connection)
display(df)

,experience,salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***
Выводы по Юниту 6.

Для начинающего дата-сайентиста нашлась всего 51 вакансия, что составляет всего 2,8% от общего числа вакансий (1771), имеющих отношение к данным. 
В среднем, для вакансии дата-сайентиста указано 6 ключевых навыков. Можно предположить, что наиболее востребованными являются Python и SQL.  
Зарплата начинающего дата сайентиста составляет около 75 тыс. рублей. Но уже через год работы она может вырасти в 2 раза.


# Общий вывод по проекту

* По количеству открытых вакансий лидируют крупные города России, а так же некоторые большие города ближнего зарубежья (Минск, Алматы). Москва в этом списке - абсолютный лидер. Это наглядно продемонстрировано на графике № 1 ниже.
* Большое количество работодателей так же сконцентрировано в крупных городах России и СНГ (см. Таблицу №1). При этом, по уровню средней зарплаты лидируют не крупные города России, как предполагалось изначально, а регионы Европы (Германия, Болгария, Испания), - Москва в этом списке занимает только 50-е место в этом списке (см. Таблицу №2). Можно предположить, что в странах Европы открыто некоторое количество вакансии для высококвалифицированных специалистов, которые могут претендовать на высокий уровень заработной платы.
* Количество открытых "удаленных" вакансий в регионах коррелирует с кол-вом открытых вакансий на "полный день/полная занятость" в этих же регионах. Т.е. чем крупнее регион, тем больше открыто в нем вакансий для работы как из офиса, так и удаленно (см. таблицы №3 и №4).

In [697]:
import pandas as pd
import plotly.express as px

query = f'''select 
                a.name "Регион",
                count(v.id) "Кол-во открытых вакансий"
            from public.AREAS as a
                left join public.VACANCIES as v on a.id = v.area_id
            group by a.name
            order by 2 desc
            limit 20           
        '''
df = pd.read_sql_query(query, connection)

fig = px.bar(
    data_frame=df,     
    x="Регион",     
    y="Кол-во открытых вакансий",
    color="Регион",       
    height=500, #высота
    width=1000, #ширина
    title='График №1. Количество открытых вакансий по регионам'
)
fig.show()
fig.write_html("plotly/bar_1.html")



In [698]:
query = f'''select 
                a.name "Регион",
                count(distinct e.id) "Кол-во работодателей"
            from public.AREAS as a
                left join public.EMPLOYERS as e on a.id=e.area
            group by a.name
            order by 2 desc
            limit 10                       
        '''
df = pd.read_sql_query(query, connection)
print("Таблица №1. Количество работодателей в регионах.\n")
display(df)

Таблица №1. Количество работодателей в регионах.



,Регион,Кол-во работодателей
0,Москва,5864
1,Санкт-Петербург,2217
2,Минск,1115
3,Алматы,721
4,Екатеринбург,609
5,Новосибирск,573
6,Казань,480
7,Нижний Новгород,426
8,Россия,410
9,Краснодар,409


In [699]:
query = f'''select 
                a.name "Регион",
                avg((coalesce(v.salary_from, v.salary_to)+coalesce(v.salary_to, v.salary_from))/2) "Средняя зарплата"
            from public.AREAS as a
                left join public.VACANCIES as v on a.id=v.area_id
            where v.salary_from is not NULL or v.salary_to is not NULL
            group by a.name
            order by 2 desc
            limit 45
        '''
df = pd.read_sql_query(query, connection)
print("Таблица №2. Средняя зарплата в регионах.\n")
display(df)

Таблица №2. Средняя зарплата в регионах.



,Регион,Средняя зарплата
0,Германия,452751.000000
1,Болгария,324348.000000
2,Испания,300000.000000
3,Черногория,288589.250000
4,Литва,263018.500000
5,Польша,257155.666667
6,Петропавловка (Республика Бурятия),250000.000000
7,Турция,235435.421053
8,Кипр,223735.589744
9,Кутаиси,220474.666667


In [700]:
query = f'''select            
                a.name "Регион вакансии",                                
                v.schedule,
                v.employment,
                count(v.id)
            from public.VACANCIES  as v
                left join public.AREAS as a on a.id=v.area_id

            where (v.schedule='Полный день' and v.employment='Полная занятость') 
            group by a.name, v.schedule, v.employment
            order by 4 desc  
            limit 15                        
        '''
df = pd.read_sql_query(query, connection)
print('Таблица №3. Количество вакансий с работой в "офисе" по регионам.')
display(df)


Таблица №3. Количество вакансий с работой в "офисе" по регионам.


,Регион вакансии,schedule,employment,count
0,Москва,Полный день,Полная занятость,3968
1,Санкт-Петербург,Полный день,Полная занятость,2088
2,Алматы,Полный день,Полная занятость,1561
3,Минск,Полный день,Полная занятость,1526
4,Новосибирск,Полный день,Полная занятость,1457
5,Екатеринбург,Полный день,Полная занятость,1292
6,Нижний Новгород,Полный день,Полная занятость,1087
7,Казань,Полный день,Полная занятость,1023
8,Краснодар,Полный день,Полная занятость,878
9,Самара,Полный день,Полная занятость,790


In [701]:
query = f'''select            
                a.name "Регион вакансии",                                
                v.schedule,
                v.employment,
                count(v.id)
            from public.VACANCIES  as v
                left join public.AREAS as a on a.id=v.area_id

            where (v.schedule='Удаленная работа' and v.employment='Полная занятость') 
            group by a.name, v.schedule, v.employment
            order by 4 desc  
            limit 15                        
        '''
df = pd.read_sql_query(query, connection)
print('Таблица №4. Количество вакансий с удаленным форматом работы по регионам.')
display(df)

Таблица №4. Количество вакансий с удаленным форматом работы по регионам.


,Регион вакансии,schedule,employment,count
0,Москва,Удаленная работа,Полная занятость,941
1,Санкт-Петербург,Удаленная работа,Полная занятость,431
2,Нижний Новгород,Удаленная работа,Полная занятость,384
3,Новосибирск,Удаленная работа,Полная занятость,328
4,Краснодар,Удаленная работа,Полная занятость,311
5,Минск,Удаленная работа,Полная занятость,309
6,Казань,Удаленная работа,Полная занятость,263
7,Екатеринбург,Удаленная работа,Полная занятость,253
8,Самара,Удаленная работа,Полная занятость,236
9,Ростов-на-Дону,Удаленная работа,Полная занятость,233


In [702]:
# Закрываем соединение после окончания работы
connection.close()